In [ ]:
# Load in the relevant packages
from ase.io import read,write
from ase.build import bulk
import numpy as np
import ase.db as db
from ase.visualize import view
from ase.calculators.lj import LennardJones
import matplotlib.pyplot as plt
import json
import sympy
import matplotlib.pyplot as plt
import os
import sys

# Get the absolute path of the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add the parent directory to the sys.path list
sys.path.insert(0, parent_dir)

# Import the module from the parent directory
from helper import fit_task

# try:
#     os.remove('G2B.db')
# except:
#     pass

#SYSTEM
damping=0.1 # Empirical damping parameter used in the rattleling, hard materials need a small value... 
            # Natural range problably around 0.25-5. 
Tm= 1465    # Boiling point!
a0=5.6   # Cell paramter

crystal=bulk('NaCl','rocksalt',cubic=True,a=a0)

#FORCE-FIELD
# Comparison of Fixed Charge and Polarizable Models
# for Predicting the Structural, Thermodynamic and Transport Properties of Molten
# Alkali Chlorides
#         ++      +-         --
# A       6.081   4.865    3.649   0.317 
# sigma   2.340   2.755    3.170 
# C      24.181 161.205 1669.618 
# D      11.515 200.066 3353.630
E_conv= 0.0433641153087705

r_cut=8.0
CCS_res=0.1

pairs=["Na-Na","Na-Cl","Cl-Cl"]
charge_dict= {"Na":1,"Cl":-1}

V_ij=" A_ij*exp( (sigma- r_ij) /rho)      -C_ij/(r_ij**6)-D_ij/(r_ij**8)"
FF= {"Na-Na": {"A_ij":6.081*E_conv, "sigma":2.340 ,"rho":0.317 , "C_ij":24.181*E_conv   , "D_ij":11.515*E_conv  }, 
     "Na-Cl": {"A_ij":4.865*E_conv, "sigma":2.755 ,"rho":0.317 , "C_ij":161.205*E_conv  , "D_ij":200.066*E_conv},
     "Cl-Cl": {"A_ij":3.649*E_conv, "sigma":3.170 ,"rho":0.317 , "C_ij":1669.618*E_conv , "D_ij":3353.630*E_conv}}



FT=fit_task(pairs=pairs,
            crystal=crystal,
            Tm=Tm,
            r_cut=r_cut,
            V_ij=V_ij,
            charge_dict=charge_dict,
            damping=damping,
            CCS_res=CCS_res
            )

FT.assign_params(FF)

# BUILD INITIAL TRAINING-SET
We first deterimine the bounds for the volume and then try to fill up data uniformly accros the assible volume range.

In [ ]:
FT.init_training()

# BUILD SCARMBLED DATA SET

In [ ]:
FT.scramble(size=1000,DB="G2B.db",damping=1.5)

# Check sampling quality

In [ ]:
FT.check_sampling()

# Fit and analyse

In [ ]:
import pandas as pd
df=pd.DataFrame()

In [ ]:
from ccs_fit.scripts.ccs_fetch import ccs_fetch
from ccs_fit import ccs_fit
import pandas as pd
import seaborn as sns

for Ns in [1,2,4,8,16,32]:
    i=0
    while i < 10:
        try:
            ccs_fetch(mode="CCS+Q",DFT_DB="G2B.db",include_forces=True,charge_dict=charge_dict,R_c=r_cut,Ns=Ns)
            ccs_fit("CCS_input.json")
            Overlap = float( FT.calculate_overlap_rmse()['Total'] )
            q_err=FT.compare_q()
            tmp=pd.DataFrame([{"Method":"CCS", "No_samples":Ns,"Overlap":Overlap,"q_error":q_err }])
            df=pd.concat([df,tmp],ignore_index=True)
            Overlap = float(FT.calculate_overlap_rmse(UNC=True)['Total'])
            q_err=FT.compare_q(UNC=True)
            tmp=pd.DataFrame([{"Method":"UNC", "No_samples":Ns,"Overlap":Overlap,"q_error":q_err }])
            df=pd.concat([df,tmp],ignore_index=True)
            i += 1    
        except:
            pass
         
    sns.lineplot(x='No_samples', y='Overlap', hue='Method', data=df)
    plt.show()
    sns.lineplot(x='No_samples', y='q_error', hue='Method', data=df)
    plt.show()



In [ ]:
import pandas as pd
df_NoF=pd.DataFrame()


In [ ]:
from ccs_fit.scripts.ccs_fetch import ccs_fetch
from ccs_fit import ccs_fit
import pandas as pd
import seaborn as sns

for Ns in [8,16,32]:
    i=0
    while i < 10:
        ccs_fetch(mode="CCS+Q",DFT_DB="G2B.db",include_forces=False,charge_dict=charge_dict,R_c=r_cut,Ns=Ns)
        ccs_fit("CCS_input.json")
        Overlap = float( FT.calculate_overlap_rmse()['Total'] )
        q_err=FT.compare_q()
        tmp=pd.DataFrame([{"Method":"CCS", "No_samples":Ns,"Overlap":Overlap,"q_error":q_err }])
        df_NoF=pd.concat([df_NoF,tmp],ignore_index=True)
        Overlap = float(FT.calculate_overlap_rmse(UNC=True)['Total'])
        q_err=FT.compare_q(UNC=True)
        tmp=pd.DataFrame([{"Method":"UNC", "No_samples":Ns,"Overlap":Overlap,"q_error":q_err }])
        df_NoF=pd.concat([df_NoF,tmp],ignore_index=True)
        i += 1    

         
    sns.lineplot(x='No_samples', y='Overlap', hue='Method', data=df_NoF)
    plt.show()
    sns.lineplot(x='No_samples', y='q_error', hue='Method', data=df_NoF)
    plt.show()